### 数据预处理

In [1]:
import pandas as pd
import numpy as np

data = pd.read_csv('user.csv')
# 删除文本属性
for col in ['所在地', '自我介绍', '标签', '昵称']:
    data = data.drop(col, 1)
        
# 删除缺失值过多的属性
for col in data.columns:
    if sum(data[col].isnull())/data.shape[0] > 0.1:
        data = data.drop(col, 1)
data.head()

,userID,年龄,所在省,creditedNum,文凭,性别,身高,学历认证,身份认证,手机认证,...,是否吸烟,是否喝酒,是否想要小孩,要求伴侣月收入,月薪,有无子女,民族,职业,要求伴侣购房情况,要求伴侣身高
0,56013059,34,河南,1,本科,男,177,0,0,1,...,不吸烟，但不反感,不喝酒,NaN,"2000以下,4000-5000,7000-10000,15000-20000,30000-...",2000-5000,没有,汉族,其它,",与父母同住,需要时购置,已购房(无贷款),住单位房,住亲朋家",163~173厘米
1,50027250,48,江苏,1,本科,男,171,0,0,1,...,社交时偶尔吸烟,兴致时小酌,想,"2000以下,4000-5000,7000-10000,15000-20000,30000-...",10000-20000,没有,汉族,高级经营管理,",与父母同住,需要时购置,已购房(无贷款),住单位房,住亲朋家",160~170厘米
2,55748374,34,河南,1,本科,男,170,0,0,1,...,不吸烟，但不反感,不喝酒,想,"2000以下,4000-5000,7000-10000,15000-20000,30000-...",5000-10000,没有,汉族,其它,",与父母同住,需要时购置,已购房(无贷款),住单位房,住亲朋家",160~168厘米
3,50030782,42,江苏,0,本科,男,174,0,0,0,...,烟不离手,兴致时小酌,还没想好,"2000以下,4000-5000,7000-10000,15000-20000,30000-...",5000-10000,没有,汉族,高级经营管理,",与父母同住,需要时购置,已购房(无贷款),住单位房,住亲朋家",162~169厘米
4,53824654,35,河北,2,本科,男,178,0,1,1,...,不吸烟，但不反感,不喝酒,想,"2000以下,4000-5000,7000-10000,15000-20000,30000-...",2000-5000,没有,汉族,高级经营管理,",与父母同住,需要时购置,已购房(无贷款),住单位房,住亲朋家",164~172厘米


In [34]:
import pickle

pairs = set()
kongjian = pickle.load(open('kongjian.pkl','rb'))
for value in kongjian.values():
    if len(value) == 0:
        continue
    try:
        user1 = value['user1']['uid']
        user2 = value['user2']['uid']
        if user1 == user2:
            continue
        pairs.add((min(user1,user2), max(user1,user2)))
    except:
        continue

In [35]:
user = pickle.load(open('man.pkl', 'rb'))
user.update(pickle.load(open('woman.pkl', 'rb')))

broke = set()
together = set()
in_love = ['已经结婚了', '正在恋爱中', '正在约会中']
for user1,user2 in pairs:

    if user1 in user and user2 in user:
        if user[user1]['status'] in in_love and user[user2]['status'] in in_love:
            together.add((user1,user2))
        else:
            broke.add((user1,user2))
    else:
        continue

In [36]:
total = len(together)+len(broke)
print('总共%d对情侣'%total)
print('%d(占比%4.2f%%)对情侣还在一起'%(len(together), len(together)/total*100))
print('%d(占比%4.2f%%)对情侣已经分手'%(len(broke), len(broke)/total*100))

总共616对情侣
364(占比59.09%)对情侣还在一起
252(占比40.91%)对情侣已经分手


In [37]:
man = dict()
woman = dict()
with open('user.csv', 'r', encoding='utf8') as f:
    lines = f.readlines()
    keys = lines[0][1:].strip().split(',')
    for line in lines[1:]:
        characters = line.strip().split(',')
        if characters[6] == '男':
            man[characters[0]] = ','.join(characters[1:])
        else:
            woman[characters[0]] = ','.join(characters[1:])
            

In [ ]:
data = []
for user1,user2 in together:
    if user1 in man and user2 in woman:
        data.append(man[user1]+woman[user2]+',1\n')
    elif user2 in man and user1 in woman:
        data.append(man[user2]+woman[user1]+',1\n')
    else:
        continue

for user1,user2 in broke:
    if user1 in man and user2 in woman:
        data.append(man[user1]+woman[user2]+',0\n')
    elif user2 in man and user1 in woman:
        data.append(man[user2]+woman[user1]+',0\n')
    else:
        continue

        
import csv
with open('data.csv', 'w', encoding='utf8', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['男_'+i for i in keys]+['女_'+i for i in keys]+['result'])
    writer.writerows(data)

In [43]:
import pandas as pd
import numpy as np

data = pd.read_csv('data.csv')
data.head()

In [ ]:
data_id = ['userID']
categorical = ['所在省', '文凭', '性别', '学历认证', '身份认证', '手机认证', '芝麻信用认证', '身份通认证', '照片', '本人婚姻状况', 
               '宗教信仰', '属相', '恋爱类型', '星座', '是否吸烟', '是否喝酒', '是否想要小孩', '有无子女', '民族', '职业', '月薪']
numerical = ['年龄', 'creditedNum', '身高']

cate_data = data[categorical]
numerical_data = data[numerical]

In [ ]:
cate_data = cate_data.fillna('missing')
cate_data = pd.get_dummies(cate_data)

In [ ]:
for col in numerical_data.columns:
    numerical_data[col] = numerical_data[col].fillna(numerical_data[col].mean())
x = pd.concat([cate_data, numerical_data], axis=1)
x.head()

### 训练模型

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# 使用交叉验证的方法，把数据集分为训练集合测试集
from sklearn.model_selection import train_test_split

from sklearn import datasets
from sklearn.linear_model import LogisticRegression

# 切分数据集
def split_data(x, y):
    # 将数据集拆分为训练集和测试集 
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.30, random_state=0)
    return X_train, X_test, y_train, y_test

# 使用LogisticRegression考察线性回归的预测能力
def test_LogisticRegression(X_train, X_test, y_train, y_test):
    # 选择模型
    cls = LogisticRegression()

    # 把数据交给模型训练
    cls.fit(X_train, y_train)

    print("Coefficients:%s, intercept %s"%(cls.coef_,cls.intercept_))
    print("Residual sum of squares: %.2f"% np.mean((cls.predict(X_test) - y_test) ** 2))
    print('Score: %.2f' % cls.score(X_test, y_test))

if __name__=='__main__':
    X_train,X_test,y_train,y_test = split_data(X) # 产生用于回归问题的数据集
    test_LogisticRegression(X_train,X_test,y_train,y_test) # 调用 test_LinearRegression